In [1]:
import os
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher
from tfx.proto import trainer_pb2, pusher_pb2

2025-07-07 22:50:19.701635: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 22:50:20.013463: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-07 22:50:20.762134: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-07 22:50:20.762591: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-07 22:50:20.899701: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
# --- GANTI DENGAN USERNAME ANDA ---
USERNAME = "raffihakim"
# -----------------------------------

PIPELINE_NAME = "student-grade-pipeline"
PIPELINE_ROOT = os.path.join(f'{USERNAME}-pipeline')
DATA_ROOT = "data"
MODULE_FILE = os.path.join("pipeline", "module.py")
SERVING_MODEL_DIR = os.path.join(PIPELINE_ROOT, "serving_model", PIPELINE_NAME)

# Inisialisasi InteractiveContext
context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [3]:
# 1. ExampleGen: Mengambil data dari CSV
example_gen = CsvExampleGen(input_base=DATA_ROOT)
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [4]:
# 2. StatisticsGen: Menghitung statistik
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)

# (Opsional) Tampilkan visualisasi statistik
context.show(statistics_gen.outputs['statistics'])

In [ ]:
# 3. SchemaGen: Membuat skema data
schema_gen = SchemaGen(statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)
context.run(schema_gen)

context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'age',INT,required,,-
'attendance_percentage',FLOAT,required,,-
'diet_quality',STRING,required,,'diet_quality'
'exam_score',FLOAT,required,,-
'exercise_frequency',INT,required,,-
'extracurricular_participation',STRING,required,,'extracurricular_participation'
'gender',STRING,required,,'gender'
'internet_quality',STRING,required,,'internet_quality'
'mental_health_rating',INT,required,,-


,Values
Domain,
'diet_quality',"'Fair', 'Good', 'Poor'"
'extracurricular_participation',"'No', 'Yes'"
'gender',"'Female', 'Male', 'Other'"
'internet_quality',"'Average', 'Good', 'Poor'"
'parental_education_level',"'Bachelor', 'High School', 'Master', 'None'"
'part_time_job',"'No', 'Yes'"


In [ ]:
# 4. ExampleValidator: Memvalidasi data
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
context.run(example_validator)

context.show(example_validator.outputs['anomalies'])

In [7]:
# 5. Transform: Melakukan feature engineering
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=MODULE_FILE
)
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying module.py -> build/lib
installing to /tmp/tmpk2g4o52u
running install
running install_lib
copying build/lib/module.py -> /tmp/tmpk2g4o52u
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmpk2g4o52u/tfx_user_code_Transform-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8.egg-info
running install_scripts


/mnt/d/proyek1/mlops/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


creating /tmp/tmpk2g4o52u/tfx_user_code_Transform-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8.dist-info/WHEEL
creating '/tmp/tmprogtuea3/tfx_user_code_Transform-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8-py3-none-any.whl' and adding '/tmp/tmpk2g4o52u' to it
adding 'module.py'
adding 'tfx_user_code_Transform-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8.dist-info/WHEEL'
adding 'tfx_user_code_Transform-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8.dist-info/top_level.txt'
adding 'tfx_user_code_Transform-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8.dist-info/RECORD'
removing /tmp/tmpk2g4o52u
Processing ./raffihakim-pipeline/_wheels/tfx_user_code_Transform-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8-py3-none-any.whl
Processing 

2025-07-07 22:55:30.922108: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-07-07 22:55:30.927721: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


INFO:tensorflow:Assets written to: raffihakim-pipeline/Transform/transform_graph/5/.temp_path/tftransform_tmp/10219551ce00482fa5b9f5e06e68b6c0/assets


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False)
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [8]:
# 6. Trainer: Melatih model
trainer = Trainer(
    module_file=MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=2000),
    eval_args=trainer_pb2.EvalArgs(num_steps=500)
)
context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying module.py -> build/lib
installing to /tmp/tmp_6lnh6il
running install
running install_lib
copying build/lib/module.py -> /tmp/tmp_6lnh6il
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmp_6lnh6il/tfx_user_code_Trainer-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8.egg-info
running install_scripts
creating /tmp/tmp_6lnh6il/tfx_user_code_Trainer-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f1

/mnt/d/proyek1/mlops/lib/python3.10/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing ./raffihakim-pipeline/_wheels/tfx_user_code_Trainer-0.0+df6e38e60de01664e993cd39238e139b9b61334c33dc28359f10e22db7ff7ad8-py3-none-any.whl
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 age (InputLayer)            [(None, 1)]                  0         []                            
                                                                                                  
 study_hours_per_day (Input  [(None, 1)]                  0         []                            
 Layer)                                                                                           
                                                                                                  
 social_media_hours (InputL  [(None, 1)]                  0         []                            
 ayer)                                      

NotFoundError: Graph execution error:

Detected at node IteratorGetNext defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/mnt/d/proyek1/mlops/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/mnt/d/proyek1/mlops/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/mnt/d/proyek1/mlops/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/mnt/d/proyek1/mlops/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 211, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/mnt/d/proyek1/mlops/lib/python3.10/site-packages/ipykernel/kernelbase.py", line 545, in

In [ ]:
# 7. Evaluator: Mengevaluasi performa model
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model']
)
context.run(evaluator)

In [ ]:
# 8. Pusher: Menyimpan model jika performanya bagus
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(base_directory=SERVING_MODEL_DIR)
    )
)
context.run(pusher)